In [4]:
from pathlib import Path
import json
from PIL import Image

In [5]:
dataset_dir = Path("..")

# Define the images directory and duplicates directory using Path objects
image_dir = dataset_dir / "images"
meta_dir = dataset_dir / "metadata"

label_dir = dataset_dir / "labels"
yolo_dir = dataset_dir / "yolo"
yolo_labels_dir = yolo_dir / "labels"
yolo_labels_dir.mkdir(exist_ok=True, parents=True)

label_all_dir = dataset_dir / "labels_all"
yolo_all_dir = dataset_dir / "yolo_all"
yolo_all_labels_dir = yolo_all_dir / "labels"
yolo_all_labels_dir.mkdir(exist_ok=True, parents=True)

# label_tiny_dir = dataset_dir / 'labels_tiny'
# yolo_tiny_dir = dataset_dir / 'yolo_tiny'
# yolo_tiny_labels_dir = yolo_tiny_dir / 'labels'
# yolo_tiny_labels_dir.mkdir(exist_ok=True, parents=True)

label_background_dir = dataset_dir / "labels_background"

# List all objects in the image directory
objs = sorted([obj.name for obj in image_dir.iterdir()])

with open(meta_dir / "id_to_name.json", "r") as f:
    id_to_name = json.load(f)
with open(meta_dir / "id_to_name_gen.json", "r") as f:
    id_to_name_gen = json.load(f)

with open(meta_dir / "near_duplicates.json", "r") as f:
    near_duplicates = json.load(f)

with open(label_dir / "labels_nms.json", "r") as f:
    labels_nms = json.load(f)
with open(label_all_dir / "no_ann.json", "r") as f:
    no_ann_all = json.load(f)
with open(label_all_dir / "labels_nms.json", "r") as f:
    labels_nms_all = json.load(f)
# with open(label_tiny_dir/'no_ann.json', 'r') as f:
#     no_ann_tiny = json.load(f)
# with open(label_tiny_dir/'labels_nms.json', 'r') as f:
#     labels_nms_tiny = json.load(f)

In [6]:
with open(meta_dir / "classes.json", "r") as f:
    class_dict = json.load(f)

for k, v in class_dict.items():
    print(f"  {k}: {v}")

  articulated dump truck: 0
  bulldozer: 1
  combined piling and drilling rig: 2
  crawler crane: 3
  crawler excavator: 4
  crawler loader: 5
  duty cycle crane: 6
  gantry crane: 7
  log loader: 8
  maritime crane: 9
  material handling machine: 10
  mining bulldozer: 11
  mining excavator: 12
  mining truck: 13
  mobile crane: 14
  pipelayer: 15
  pontoon excavator: 16
  reachstacker: 17
  telescopic handler: 18
  tower crane: 19
  truck mixer: 20
  wheel excavator: 21
  wheel loader: 22


## convert original images to yolo format and save them (no need to run again)

In [ ]:
# convert ORIGINAL image annotations to yolo format (no need to run again)
# no need to run again, done in covert_to_yolo.ipynb
image_list = []
for id, ann in labels_nms.items():
    if ann["boxes"] == []:  # Skip images with no annotations
        continue
    obj = id_to_name[id + ".jpg"].split("/")[0]
    image_list.append(f"./images/{obj}/{id}.jpg")
    boxes_xywh = ann["boxes"]
    classes = ann["classes"]
    cls_ids = [class_dict[class_name] for class_name in classes]

    txt = ""
    # yolo format: "class_id x y w h"
    for i in range(len(boxes_xywh)):
        cls_id = cls_ids[i]
        box = boxes_xywh[i]
        txt += f"{cls_id} {box[0]:.6f} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f}\n"

    # save
    (yolo_labels_dir / obj).mkdir(exist_ok=True, parents=True)
    with open(yolo_labels_dir / obj / (id + ".txt"), "w") as f:
        f.write(txt)

print(len(image_list))

In [ ]:
# # convert groundingdino-tiny's image annotations to yolo format (no need to run again)

# image_list = []
# for id, ann in labels_nms_tiny.items():
#     if ann['boxes'] == []: # Skip images with no annotations
#         continue
#     obj = id_to_name[id+'.jpg'].split('/')[0]
#     image_list.append(f'./images/{obj}/{id}.jpg')
#     boxes_xywh = ann['boxes']
#     classes = ann['classes']
#     cls_ids = [class_dict[class_name] for class_name in classes]

#     txt = ''
#     # yolo format: "class_id x y w h"
#     for i in range(len(boxes_xywh)):
#         cls_id = cls_ids[i]
#         box = boxes_xywh[i]
#         txt += f"{cls_id} {box[0]:.6f} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f}\n"

#     # save
#     (yolo_tiny_labels_dir /obj).mkdir(exist_ok=True, parents=True)
#     with open(yolo_tiny_labels_dir /obj/ (id+'.txt'), 'w') as f:
#         f.write(txt)

# print(len(image_list))

In [7]:
# convert ALL image annotations to yolo format (no need to run again)

image_list = []

for id, ann in labels_nms_all.items():
    if ann["boxes"] == []:  # Skip images with no annotations
        continue
    if id[0] == "d":  # for dreambooth generated images
        obj = id_to_name_gen[id + ".jpg"].split("/")[1].replace("_", " ")
    else:
        obj = id_to_name[id + ".jpg"].split("/")[0]
    image_list.append(f"./images/{obj}/{id}.jpg")
    boxes_xywh = ann["boxes"]
    classes = ann["classes"]
    cls_ids = [class_dict[class_name] for class_name in classes]

    txt = ""
    # yolo format: "class_id x y w h"
    for i in range(len(boxes_xywh)):
        cls_id = cls_ids[i]
        box = boxes_xywh[i]
        txt += f"{cls_id} {box[0]:.6f} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f}\n"

    # save
    (yolo_all_labels_dir / obj).mkdir(exist_ok=True, parents=True)
    with open(yolo_all_labels_dir / obj / (id + ".txt"), "w") as f:
        f.write(txt)

print(len(image_list))

83485


In [8]:
assert len(image_list) == (len(labels_nms_all) - len(no_ann_all))

## switch class id for lvis

In [10]:
# the most similar class is given by Sentence-BERT
lp_to_lvis = {
    0: 1122,
    1: 163,
    2: 395,
    3: 554,
    4: 841,
    5: 894,
    6: 469,
    7: 556,
    8: 652,
    9: 51,
    10: 932,
    11: 163,
    12: 841,
    13: 1122,
    14: 554,
    15: 810,
    16: 841,
    17: 1068,
    18: 1074,
    19: 1169,
    20: 695,
    21: 841,
    22: 1177,
}

In [11]:
lvis_yolo_dir = dataset_dir / "yolo_lvis_label"
lvis_yolo_labels_dir = lvis_yolo_dir / "labels"
image_list = []

for path in yolo_labels_dir.rglob("*.txt"):
    obj = path.parent.name
    id = path.stem
    # append all images
    image_list.append(f"./images/{obj}/{id}.jpg")

    with open(path, "r") as f:
        txt_list = f.readlines()
    new_txt = ""
    for txt in txt_list:
        cls_id, x, y, w, h = txt.split()
        cls_id = int(cls_id)
        new_cls_id = lp_to_lvis[cls_id]
        new_txt += f"{new_cls_id} {x} {y} {w} {h}\n"
    lvis_label_path = lvis_yolo_labels_dir / obj / path.name
    lvis_label_path.parent.mkdir(exist_ok=True, parents=True)
    with open(lvis_label_path, "w") as f:
        f.write(new_txt)

# print(len(image_list))

with open(lvis_yolo_dir / "all.txt", "w") as f:
    for img in image_list:
        f.write(img + "\n")